__AUTHORS__:
  - Théo Perinet (22172 - theo.perinet)
  - Mathieu Rivier (23553 - mathieu.rivier)
  - Marc Monteil (23742 - marc.monteil)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BigData---Final-Project" data-toc-modified-id="BigData---Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BigData - Final Project</a></span><ul class="toc-item"><li><span><a href="#Loading-The-Data" data-toc-modified-id="Loading-The-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading The Data</a></span></li><li><span><a href="#Exploring-The-Data" data-toc-modified-id="Exploring-The-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Exploring The Data</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Analysis</a></span></li></ul></li></ul></div>

# BigData - Final Project

###### To Use when you are on google collab
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz  
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

###### TO USE WHEN YOU ARE ON GOOGLE COLLAB
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
import findspark
findspark.init()

from google.colab import drive
drive.mount('/content/drive')

## Loading The Data

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_application_name = "WannaFlop_Project"

In [3]:
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/22 17:07:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/22 17:07:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Exploring The Data

In [4]:
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField
from pyspark.sql.functions import desc
import pyspark.sql.functions as func

In [5]:
class read_info(object):
    def __init__(self, file_path, header=False, delimiter=';', schema=None):
        self.file_path = file_path
        self.header = header
        self.delimiter = delimiter
        self.schema = schema

        self.df = self._load_df()

        #self.df_abstract = self._get_df_abstract()

    def __repr__(self):
        return f"{self._nb_rows()} \n{self.df.printSchema()} \n{self.get_df_abstract()}\n {self.show_missing()}\n{self._get_stats()}"

    def show_missing(self):
        print("Missing Data per column:")
        self._count_missing().show()

    def _get_num_cols(self):
        num_cols = [
            f.name for f in self.df.schema.fields
            if isinstance(f.dataType, DoubleType) or
            isinstance(f.dataType, IntegerType)
        ]
        
        return num_cols
    def _get_rounded_df(self):
        rounded_df = self.df
        dbl_cols = self._get_num_cols()
        for col in dbl_cols:
            rounded_df = rounded_df.withColumn(col, func.round('high'))

        return rounded_df

    def get_df_abstract(self):
        rounded_df = self._get_rounded_df()

        # First 40 rows
        print("First 40 rows:")
        rounded_df.show(40)

        # Last 40 rows
        print("Last 40 rows:")
        rounded_df = rounded_df.withColumn(
            "index", monotonically_increasing_id()
        )
        rounded_df.orderBy(desc("index")).drop("index").show(40)

    def _get_periodicity(self):
        self.df['data'][0]

    def _nb_rows(self):
        # Number of total rows
        print("Number of rows: " + str(self.df.count()) + "\n")

    def _handle_csv(self):
        '''
        @description: Read the csv file and return a Spark DataFrame

        @arg csv_file_path: Path to the csv file
        @arg header: boolean whether to load a header or not
        @arg delimiter: which delimiter to use by default
        '''
        return spark.read.option("inferSchema", "true").option("nullValue", "null").csv(
            self.file_path,
            sep=self.delimiter,
            schema=self.schema,
            header=self.header,
        )
    
    def _handle_json(self):
        return spark.read.json(self.file_path)

    def _load_df(self):
        ####### ADD TRY CATCH #####
        extension = self.file_path.split(".")[-1]

        df = None
        if extension == 'json':
            df = self._handle_json()
        elif extension == 'csv':
            df = self._handle_csv()

        return df

    def _count_missing(self):
        cols = self.df.columns
        cols.remove('Date')
        return self.df.select(
            [
                count(when(isnan(c) | col(c).isNull(), c)).alias(c)
                for c in cols
            ]
        )
        #.show()
        
    def _get_stats(self):
        self.df.summary().show()

In [6]:
print(h)
h.remove('Date')
print(h)

NameError: name 'h' is not defined

In [ ]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', IntegerType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])
AMZN = read_info('stocks_data/AMAZON.csv', header=True, delimiter=',', schema=amzn_schema)

In [ ]:
AMZN = read_info('stocks_data/AMAZON.csv', header=True, delimiter=',', schema=amzn_schema)

In [ ]:
print(AMZN)

In [ ]:
##### A FAIRE !!!! UN SCHEMA !!!!!

In [ ]:
AMZN.get_df_abstract()

In [ ]:
AMZN.show_missing()

In [ ]:
AMZN._get_stats()

In [ ]:
AMZN.df.withColumn("test", 
              func.datediff(AMZN.df["date"][0], AMZN.df["date"][1])).show()

In [ ]:
AMZN.df["Date"].getItem(2)

In [ ]:
AMZN.df.first()['Date']

In [ ]:
AMZN.df.__get_item(0)

In [ ]:
AMZN.df.second()['Date']

In [ ]:
func.getrows(AMZN.df, rownums=[0, 2]).collect()

In [ ]:
AMZN.df[0].__getitem__("Date").first()

In [ ]:
AMZN.df[0]

In [ ]:
AMZN.df.select('Date').show()

In [ ]:
AMZN.df


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window



my_window = Window.partitionBy().orderBy("Date")

df = AMZN.df.withColumn("prev_value", F.lag(AMZN.df.Date).over(my_window))
df = df.withColumn("diff", F.when(F.isnull(F.datediff(df.Date, df.prev_value)), 0)
                              .otherwise(F.datediff(df.Date, df.prev_value)))

In [ ]:
df.select("diff").show()

In [ ]:
from pyspark.sql.functions import mean

In [ ]:
df.select(mean('diff')).first()[0]

In [ ]:
AMZN.df.stat.corr('High', 'Low')

TODO: Create function to compute per month week year

In [ ]:
AMZN.df.select(mean ("Close")).first()[0]

In [ ]:
def get_col_mean(df, col):
    return df.select(mean (col)).first()[0]

In [ ]:
get_col_mean(AMZN.df, "Close")

In [ ]:
AMZN.df.groupBy(func.weekofyear("day").alias("date_by_week")).agg(sum("Date"))

In [ ]:
AMZN.df.groupBy(func.weekofyear("day").alias("date_by_week")).agg(sum("Close")).orderBy("date_by_week").show()

In [ ]:
AMZN.df.withColumn("Date",func.date_sub(func.next_day(col("Date"),"sunday"),7)).groupBy("Date").agg(sum("Close").cast("int").alias("Close_total")).orderBy("week_strt_day").show()


In [ ]:
AMZN.df.groupBy("Date").select("Close").show()

In [ ]:
AMZN.df.groupBy(func.month("Date").alias("hour")).agg(mean("Close").alias("close_mean")).show()

In [ ]:
AMZN.df.groupBy(func.year("Date").alias("hour")).agg(mean("Close").alias("close_mean")).show()

In [ ]:
get_avg(AMZN.df, "Close", func.year)

In [ ]:
get_avg(AMZN.df, "Open", func.year)

In [ ]:
class Exploration(object):
    def __init__(self, df):
        self.df = df

    def get_oc_avg(self, fun):
        close = self._compute_avg(self.df, "Close", fun)
        opening = self._compute_avg(self.df, "Open", fun)

        return close.join(
            opening, opening.Open_new_time == close.Close_new_time, "inner"
        ).orderBy("Close_new_time").select(
            close.Close_new_time, close.Close_mean, opening.Open_mean
        )

    def _compute_avg(self, df, col, fun):
        return df.groupBy(fun("Date").alias(col + "_new_time")).agg(
            mean(col).alias(col + "_mean")
        )

In [72]:
exAMZN = Exploration(AMZN.df)

NameError: name 'Exploration' is not defined

In [ ]:
exAMZN.get_oc_avg(func.month).show()

In [7]:
exAMZN.get_oc_avg(func.year).show()

NameError: name 'exAMZN' is not defined

In [8]:
def get_price_change(period=None):
    df = AMZN.df
    if period:
        df= exAMZN.get_oc_avg(period)
   
    return  df.withColumn('diff', ( df['Close_mean'] - df['Open_mean'] ))

In [9]:
get_price_change(func.month).show()

NameError: name 'AMZN' is not defined

In [10]:
get_price_change(func.year).show()

NameError: name 'AMZN' is not defined

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from stock import Stock

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/22 18:59:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/22 18:59:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField

In [4]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', DoubleType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

In [5]:
AMZN = Stock('stocks_data/APPLE.csv', header=True, delimiter=',', schema=amzn_schema)

In [6]:
AMZN.explore

root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

First 40 rows:


+----------+----+----+----+-----+----------+---------+------------+
|      Date|High| Low|Open|Close|    Volume|Adj Close|company_name|
+----------+----+----+----+-----+----------+---------+------------+
|2017-01-03|29.0|29.0|29.0| 29.0|1.151276E8|     27.0|       APPLE|
|2017-01-04|29.0|29.0|29.0| 29.0| 8.44724E7|     27.0|       APPLE|
|2017-01-05|29.0|29.0|29.0| 29.0| 8.87744E7|     27.0|       APPLE|
|2017-01-06|30.0|29.0|29.0| 29.0|1.270076E8|     28.0|       APPLE|
|2017-01-09|30.0|29.0|29.0| 30.0|1.342476E8|     28.0|       APPLE|
|2017-01-10|30.0|30.0|30.0| 30.0| 9.78484E7|     28.0|       APPLE|
|2017-01-11|30.0|30.0|30.0| 30.0|1.103544E8|     28.0|       APPLE|
|2017-01-12|30.0|30.0|30.0| 30.0|1.083448E8|     28.0|       APPLE|
|2017-01-13|30.0|30.0|30.0| 30.0|1.044476E8|     28.0|       APPLE|
|2017-01-17|30.0|30.0|30.0| 30.0|1.377592E8|     28.0|       APPLE|
|2017-01-18|30.0|30.0|30.0| 30.0|  9.4852E7|     28.0|       APPLE|
|2017-01-19|30.0|30.0|30.0| 30.0|1.023892E8|    

AttributeError: 'Explore' object has no attribute '_nb_rows'

In [7]:
AMZN.explore._nb_rows()

AttributeError: 'Explore' object has no attribute '_nb_rows'

In [ ]:
AMZN.explore._print_schema()

In [ ]:
AMZN.explore.get_df_abstract()

In [ ]:
AMZN.explore.get_missing()

In [115]:
AMZN.explore.get_stats()

Stock Stats:
+-------+-----+-----+-----+-----+------------+---------+------------+
|summary| High|  Low| Open|Close|      Volume|Adj Close|company_name|
+-------+-----+-----+-----+-----+------------+---------+------------+
|  count|987.0|987.0|987.0|987.0|       987.0|    987.0|         987|
|   mean| 57.0| 56.0| 57.0| 57.0|1.26770017E8|     55.0|        null|
| stddev| 24.0| 23.0| 24.0| 23.0| 6.0439452E7|     24.0|        null|
|    min| 29.0| 29.0| 29.0| 29.0|   2.01978E7|     27.0|       APPLE|
|    25%| 42.0| 41.0| 41.0| 41.0|   8.67032E7|     39.0|        null|
|    50%| 48.0| 48.0| 48.0| 48.0|  1.110428E8|     46.0|        null|
|    75%| 66.0| 65.0| 65.0| 66.0|  1.499814E8|     64.0|        null|
|    max|138.0|131.0|138.0|134.0|    4.4794E8|    134.0|       APPLE|
+-------+-----+-----+-----+-----+------------+---------+------------+



## Analysis

In [ ]:
AMZN.analysis.get_oc_avg("day").show()

In [ ]:
AMZN.analysis.get_oc_avg("year").show()

In [116]:
AMZN.analysis.get_price_change("month").show()

+-----------------+--------------------+
|Date_period_month|        price_change|
+-----------------+--------------------+
|          2017-01|  1.3874988555908203|
|          2017-02|   2.490001678466797|
|          2017-03|  1.4425010681152344|
|          2017-04|-0.01500320434570...|
|          2017-05|  1.9149971008300781|
|          2017-06| -2.2874984741210938|
|          2017-07|  0.9624977111816406|
|          2017-08|  3.7249984741210938|
|          2017-09|  -2.670001983642578|
|          2017-10|  3.6949996948242188|
|          2017-11| 0.49500274658203125|
|          2017-12|-0.18000030517578125|
|          2018-01| -0.6825027465820312|
|          2018-02|   2.737499237060547|
|          2018-03| -2.6899986267089844|
|          2018-04|  -0.345001220703125|
|          2018-05|   5.114997863769531|
|          2018-06|  -0.720001220703125|
|          2018-07|  1.6174964904785156|
|          2018-08|               7.125|
+-----------------+--------------------+
only showing top

In [ ]:
AMZN.analysis.get_price_change("year").show()

In [ ]:
AMZN.analysis._compute_avg(AMZN.df, "Close", "month").show()

In [ ]:
AMZN.analysis.get_daily_return(period="month").show()

In [ ]:
AMZN.analysis.get_daily_return(period="day").show()

In [ ]:
AMZN.analysis.get_daily_return_rate(period="month", start_price=15.4, nb_shares=100).show()

In [ ]:
AMZN.analysis.get_daily_return_rate(period="year", start_price=15.4, nb_shares=100).show()

In [ ]:
import glob

glob.glob("stocks_data/*.csv")

In [117]:
from stocks import Stocks
stocks = Stocks(schema=amzn_schema)

In [118]:
stocks = Stocks(schema=amzn_schema)

In [ ]:
stocks.get_max_daily_return()

22/05/22 18:22:37 ERROR Executor: Exception in task 0.0 in stage 95.0 (TID 70)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2017-01-03,789.6300048828125,775.7999877929688,778.8099975585938,786.1400146484375,1657300,786.1400146484375,GOOGLE' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql

In [ ]:
AMZN.df.select('Date').show()

In [ ]:
from pyspark.sql.functions import lit,row_number,col
from pyspark.sql.window import Window

w = Window().partitionBy(lit('Date')).orderBy(lit('Date'))

df1 = AMZN.df.withColumn("row_num", row_number().over(w))

df1.filter(col("row_num").between(1,2)).show()

In [ ]:
AMZN.df.show()

In [119]:
from pyspark.sql.window import Window


SECS_IN_DAY = 86400
period = period * SECS_IN_DAY

df = AMZN.df.withColumn('Date', AMZN.df.Date.cast('timestamp'))
window = Window().partitionBy(lit('Date')).orderBy(F.col("Date").cast('long')).rangeBetween(-period, 0)

df = df.withColumn('moving_average', func.avg("Close").over(window)).select('Date', 'moving_average')

NameError: name 'period' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 582


In [120]:
df.show()

+-------------------+-----------------+
|               Date|   moving_average|
+-------------------+-----------------+
|2017-01-03 00:00:00|753.6699829101562|
|2017-01-04 00:00:00|755.4249877929688|
|2017-01-05 00:00:00|763.7666625976562|
|2017-01-06 00:00:00|771.8224945068359|
|2017-01-09 00:00:00|   776.8419921875|
|2017-01-10 00:00:00|780.0183308919271|
|2017-01-11 00:00:00| 787.576670328776|
|2017-01-12 00:00:00|796.9866739908854|
|2017-01-13 00:00:00|803.1016743977865|
|2017-01-17 00:00:00|807.0840087890625|
|2017-01-18 00:00:00|            809.4|
|2017-01-19 00:00:00|811.4039916992188|
|2017-01-20 00:00:00|   810.3419921875|
|2017-01-23 00:00:00| 810.489990234375|
|2017-01-24 00:00:00|812.4816589355469|
|2017-01-25 00:00:00|816.9483337402344|
|2017-01-26 00:00:00|822.2266743977865|
|2017-01-27 00:00:00|826.6816813151041|
|2017-01-30 00:00:00| 830.356679280599|
|2017-01-31 00:00:00|831.2900085449219|
+-------------------+-----------------+
only showing top 20 rows



In [121]:
AMZN.df.show()

+----------+------------------+------------------+------------------+------------------+----------+------------------+------------+
|      Date|              High|               Low|              Open|             Close|    Volume|         Adj Close|company_name|
+----------+------------------+------------------+------------------+------------------+----------+------------------+------------+
|2017-01-03|29.082500457763672|28.690000534057617|28.950000762939453|29.037500381469727|1.151276E8|27.277639389038086|       APPLE|
|2017-01-04|29.127500534057617|           28.9375|28.962499618530273|  29.0049991607666| 8.44724E7|27.247108459472656|       APPLE|
|2017-01-05| 29.21500015258789|28.952499389648438|28.979999542236328| 29.15250015258789| 8.87744E7| 27.38566780090332|       APPLE|
|2017-01-06|29.540000915527344| 29.11750030517578| 29.19499969482422|29.477500915527344|1.270076E8| 27.69097137451172|       APPLE|
|2017-01-09|29.857500076293945|29.485000610351562|29.487499237060547|29.7474

In [123]:
AMZN.analysis.moving_average(col="Close")

+----------+--------------------+
|      Date|moving_average_Close|
+----------+--------------------+
|2017-01-03|  29.037500381469727|
|2017-01-04|  29.021249771118164|
|2017-01-05|   29.06499989827474|
|2017-01-06|   29.16812515258789|
|2017-01-09|  29.345624923706055|
|2017-01-10|  29.538750171661377|
|2017-01-11|  29.735000133514404|
|2017-01-12|   29.81874990463257|
|2017-01-13|  29.806999969482423|
|2017-01-17|  29.857500076293945|
|2017-01-18|  29.919166564941406|
|2017-01-19|  29.980833053588867|
|2017-01-20|   29.98562479019165|
|2017-01-23|   29.99062490463257|
|2017-01-24|  29.989375114440918|
|2017-01-25|  30.120625019073486|
|2017-01-26|  30.241875171661377|
|2017-01-27|  30.290999984741212|
|2017-01-30|  30.462499618530273|
|2017-01-31|   30.42937469482422|
+----------+--------------------+
only showing top 20 rows



In [18]:
from pyspark.sql.types import TimestampType
df = AMZN.df
dates = ("2017-01-03",  "2017-01-06")

date_from, date_to = [func.to_date(lit(s)) for s in dates]
import pyspark.sql.functions as func

#df = df.select(func.to_date(df.my_col).alias("time"))
sf = df.filter(df.Date > date_from ).filter(df.Date < date_to)

In [19]:
sf.show()

+----------+------------------+------------------+------------------+-----------------+------+------------------+------------+
|      Date|              High|               Low|              Open|            Close|Volume|         Adj Close|company_name|
+----------+------------------+------------------+------------------+-----------------+------+------------------+------------+
|2017-01-04|29.127500534057617|           28.9375|28.962499618530273| 29.0049991607666|  null|27.247108459472656|       APPLE|
|2017-01-05| 29.21500015258789|28.952499389648438|28.979999542236328|29.15250015258789|  null| 27.38566780090332|       APPLE|
+----------+------------------+------------------+------------------+-----------------+------+------------------+------------+



In [30]:
nb_days = 30
start_date = "2017-01-03"
df = AMZN.df
period_days = {"month": 30, "year": 365}
if not nb_days:
    nb_days = period_days[period]

nb_days += 1 # For the exclusive end
date_from = func.to_date(lit(start_date))
date_from = func.date_add(date_from, -1)


#date_to = 
df = df.filter(df.Date > date_from).filter(df.Date < func.date_add(date_from, 30))

In [31]:
df.show()

+----------+------------------+------------------+------------------+------------------+------+------------------+------------+
|      Date|              High|               Low|              Open|             Close|Volume|         Adj Close|company_name|
+----------+------------------+------------------+------------------+------------------+------+------------------+------------+
|2017-01-03|29.082500457763672|28.690000534057617|28.950000762939453|29.037500381469727|  null|27.277639389038086|       APPLE|
|2017-01-04|29.127500534057617|           28.9375|28.962499618530273|  29.0049991607666|  null|27.247108459472656|       APPLE|
|2017-01-05| 29.21500015258789|28.952499389648438|28.979999542236328| 29.15250015258789|  null| 27.38566780090332|       APPLE|
|2017-01-06|29.540000915527344| 29.11750030517578| 29.19499969482422|29.477500915527344|  null| 27.69097137451172|       APPLE|
|2017-01-09|29.857500076293945|29.485000610351562|29.487499237060547|29.747499465942383|  null|27.944602

In [29]:
sf.first()['Open']

28.962499618530273

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

from stock import Stock
from stocks import Stocks

from stock import Stock
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as func
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, StructType,StructField
from pyspark.sql import SparkSession

import pyspark.sql.functions as func
from pyspark.sql.functions import (
    col,
    isnan,
    when,
    count,
    mean,
    lag,
    isnull,
    datediff,
    asc,
    desc,
    monotonically_increasing_id,
    lit,
)
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    DateType,
    StructType,
    StructField,
)


from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window

In [16]:
amzn_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', IntegerType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])



In [17]:
stocks = Stocks(schema=amzn_schema)

In [18]:
AMZN = Stock('stocks_data/APPLE.csv', header=True, delimiter=',', schema=amzn_schema)

In [19]:
AMZN.analysis.get_window_return_rate('2017-01-03', period="month")

0.46262058281582685

In [20]:
from stocks import Stocks
stocks = Stocks(schema=amzn_schema)

In [21]:
stocks.max_period_return_rate(start_date="2017-01-03")

22/05/22 19:35:17 ERROR Executor: Exception in task 0.0 in stage 5.0 (TID 4)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2017-01-03,789.6300048828125,775.7999877929688,778.8099975585938,786.1400146484375,1657300,786.1400146484375,GOOGLE' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.c

Py4JJavaError: An error occurred while calling o271.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 4) (10.0.0.25 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2017-01-03,789.6300048828125,775.7999877929688,778.8099975585938,786.1400146484375,1657300,786.1400146484375,GOOGLE' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$

In [25]:
stocks.stocks[1].df.show()

22/05/22 19:35:41 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 7)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2017-01-03,117.83999633789062,115.51000213623047,116.02999877929688,116.86000061035156,20663900,116.86000061035156,FACEBOOK' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spa

Py4JJavaError: An error occurred while calling o183.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 7) (10.0.0.25 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2017-01-03,117.83999633789062,115.51000213623047,116.02999877929688,116.86000061035156,20663900,116.86000061035156,FACEBOOK' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:918)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelp